In [1]:
import pandas as pd

# Connection

In [2]:
from connector import connection

conn = connection()

cur = conn.cursor()

cur.execute("SELECT version();")

db_version = cur.fetchone()
print("Connected to:", db_version)



Connected to: ('PostgreSQL 16.3 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',)


# Fetching Tables

In [3]:
cur.execute("select store_id,total_amount,created_at from sales_invoices;")
sc = cur.fetchall()

sales_invoices = pd.DataFrame(sc, columns=['store_id','sc','created_at'])

In [4]:
cur.execute("select store_id,total_amount,created_at from sales_return;")
sr = cur.fetchall()

sales_return = pd.DataFrame(sr, columns=['store_id','sr','created_at'])

In [5]:
cur.execute("select id,name,store_type from stores;")
store = cur.fetchall()

stores = pd.DataFrame(store, columns=['store_id','name','type'])

# Processing

In [6]:
import datetime
sales_invoices['created_at'] = pd.to_datetime(sales_invoices['created_at'], format='%Y-%m-%d')
sales_return['created_at'] = pd.to_datetime(sales_invoices['created_at'], format='%Y-%m-%d')

In [7]:
sales_invoices['year'] = sales_invoices['created_at'].dt.year
sales_invoices['month'] = sales_invoices['created_at'].dt.month


sales_return['year'] = sales_return['created_at'].dt.year
sales_return['month'] = sales_return['created_at'].dt.month

In [8]:
sales_invoices = sales_invoices.groupby(['store_id','created_at','month','year'])['sc'].sum().reset_index()

sales_return = sales_return.groupby(['store_id','created_at','month','year'])['sr'].sum().reset_index()

# Merging

In [9]:
temp = sales_invoices.merge(sales_return, how = 'left', left_on=['store_id','created_at','month','year'], right_on = ['store_id','created_at','month','year'])

In [10]:
temp = temp.merge(stores, how = 'left', left_on=['store_id'], right_on = ['store_id'])

In [11]:
temp

,store_id,created_at,month,year,sc,sr,name,type
0,23,2024-07-18 12:10:34,7,2024,195.40,NaN,MEDKART A PARIMAL,COCO
1,23,2024-08-17 10:21:23,8,2024,2282.00,NaN,MEDKART A PARIMAL,COCO
2,23,2024-08-24 12:02:09,8,2024,4836.60,NaN,MEDKART A PARIMAL,COCO
3,23,2024-08-29 17:09:28,8,2024,134.40,NaN,MEDKART A PARIMAL,COCO
4,23,2024-08-29 17:30:23,8,2024,1080.00,NaN,MEDKART A PARIMAL,COCO
...,...,...,...,...,...,...,...,...
1410,141,2024-06-21 18:12:50,6,2024,225.10,NaN,NILKANTH PHARMA,FOFO
1411,141,2024-06-23 08:51:07,6,2024,1744.06,NaN,NILKANTH PHARMA,FOFO
1412,141,2024-06-24 15:19:15,6,2024,885.00,NaN,NILKANTH PHARMA,FOFO
1413,141,2024-06-24 16:20:55,6,2024,163.60,NaN,NILKANTH PHARMA,FOFO


# COCO report

In [12]:
temp1 = temp[(temp['type'] == 'COCO')]

### Assign Month and year

In [13]:
today = datetime.datetime.now()
m = today.month-1 # For now we'll do - 1 as no data for feb 2025
y = today.year
temp1 = temp1[(temp1['year'] == y) & (temp1['month'] == m)].reset_index(drop=True)

### Melting and unstacking

In [14]:
melted_df = pd.melt(temp1, 
                    id_vars=['store_id','name', 'created_at'], 
                    value_vars=['sc', 'sr'],
                    var_name='category', 
                    value_name='Amount')

In [15]:
temp_melted = melted_df.sort_values(['created_at'])

In [16]:
temp_melted

,store_id,name,created_at,category,Amount
0,36,MEDKART B AKOTA,2025-01-06 16:36:36,sc,888.80
35,36,MEDKART B AKOTA,2025-01-06 16:36:36,sr,NaN
1,36,MEDKART B AKOTA,2025-01-08 16:04:12,sc,70.00
36,36,MEDKART B AKOTA,2025-01-08 16:04:12,sr,NaN
2,36,MEDKART B AKOTA,2025-01-10 17:56:17,sc,580.00
...,...,...,...,...,...
32,36,MEDKART B AKOTA,2025-01-31 13:08:57,sc,27.78
68,36,MEDKART B AKOTA,2025-01-31 14:47:12,sr,NaN
33,36,MEDKART B AKOTA,2025-01-31 14:47:12,sc,27.00
34,36,MEDKART B AKOTA,2025-01-31 16:18:21,sc,27.00


In [17]:
temp_melted_1 = temp_melted.groupby(
        ['store_id', 'name','created_at','category']
    ).agg({'Amount': 'sum'}).unstack([2,3]).reset_index()
temp_melted_1 = temp_melted_1.sort_index(axis=1, level=1)

In [18]:
temp_melted_1

Amount                                \
created_at 2025-01-06 16:36:36      2025-01-08 16:04:12        
category                    sc   sr                  sc   sr   
0                        888.8  0.0                70.0  0.0   

                                                              \
created_at 2025-01-10 17:56:17      2025-01-13 10:39:18        
category                    sc   sr                  sc   sr   
0                        580.0  0.0              1175.0  0.0   

                                     ...                           \
created_at 2025-01-16 12:50:29       ... 2025-01-30 17:26:33        
category                    sc   sr  ...                  sc   sr   
0                        102.0  0.0  ...                67.0  0.0   

                                                              \
created_at 2025-01-31 13:08:57      2025-01-31 14:47:12        
category                    sc   sr                  sc   sr   
0                        27.78  0.0                27.0  0.0   

                                                name store_id  
created_at 2025-01-31 16:18:21                   NaT      NaT  
category                    sc   sr                            
0                         27.0  0.0  MEDKART B AKOTA       36  

[1 rows x 72 columns]

In [19]:
temp_melted_1.to_csv(f'salesdetail_{today.day}_{today.month}_{today.year}.csv')

# FOFO REPORT

In [20]:
temp1 = temp[(temp['type'] == 'FOFO')]

In [21]:
temp1 = temp1.fillna(0)

### Assign current month and year

In [22]:

today = datetime.datetime.now()
m = today.month
y = today.year
temp1 = temp1[(temp1['year'] == y) & (temp1['month'] == m-1)].reset_index(drop=True)

In [23]:

temp1['BaseValue'] = temp1['sc'] - temp1['sr']

In [24]:
temp1

,store_id,created_at,month,year,sc,sr,name,type,BaseValue
0,52,2025-01-06 15:36:07,1,2025,320.0,0.0,MAHALAXMI ENTERPRISE NIKOL,FOFO,320.0


In [25]:
final_temp = temp1.groupby(['name'])['BaseValue'].sum().reset_index()

final_temp['RL'] = final_temp['BaseValue'] * 0.02

In [26]:
final_temp

,name,BaseValue,RL
0,MAHALAXMI ENTERPRISE NIKOL,320.0,6.4


In [27]:
final_temp.to_csv(f'rl_{today.strftime('%B')}-{today.day}.csv')